<a href="https://colab.research.google.com/github/bam396/nyu-python-public-policy/blob/master/HW4_old.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HOMEWORK 4 Coding: Analyzing and visualizing specific categories of 311 complaints

Create a new Google Colab notebook called "HW4".

**Problem Statement:** When comparing total 311 request counts per community district against the respective population, there seems to be a general trend that the higher the population, the higher the number of 311 requests. However, there is a lot of variance in the data. Maybe narrowing in on specific complaint type categories will help us better understand trends in 311 requests across the city.

"Noise" and "heat and hot water" were the most frequest requests in the entire 311 dataset. Let's try isolating the count of requests in each of those two categories versus all the other complaints.

**Step 1.** First you will need to prep your data to get the count of complaints in each of the three categories. 

In [0]:
import pandas as pd
from google.colab import drive

In [0]:
import matplotlib.pyplot as plt 
import numpy as np

In [0]:
# You can use pd.set_option() to make sure you see all the rows and columns in your dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
drive.mount('/content/drive')
# follow the link it generates, choose your account, and then paste in the authorization code it provides

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/My Drive/Data for Python/cleaned_311_data.csv', header='infer')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,18,21,32,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


You can update the code from lecture that gets the total count of 311 requests per CD to instead get the count of 311 requests per CD that meet the criteria for the 3 categories. You should create three seprate dataframes, one per category:
1.   complaint_type = 'Noise - Residential'
2.   complaint_type = 'HEAT/HOT WATER'
3.   Everything else: (complaint_type != 'Noise - Residential' and complaint_type != 'HEAT/HOT WATER')

In [0]:
df['complaint_type'] = df['complaint_type'].str.lower()

In [7]:
cb_counts = df.groupby('community_board').size().reset_index(name='count_of_311_requests')
print(cb_counts)

              community_board  count_of_311_requests
0               0 Unspecified                  39385
1                    01 BRONX                  27353
2                 01 BROOKLYN                  68101
3                01 MANHATTAN                  21259
4                   01 QUEENS                  60425
5            01 STATEN ISLAND                  65145
6                    02 BRONX                  20039
7                 02 BROOKLYN                  44061
8                02 MANHATTAN                  36688
9                   02 QUEENS                  35238
10           02 STATEN ISLAND                  40179
11                   03 BRONX                  29738
12                03 BROOKLYN                  66360
13               03 MANHATTAN                  51399
14                  03 QUEENS                  39240
15           03 STATEN ISLAND                  48729
16                   04 BRONX                  61086
17                04 BROOKLYN                 

In [0]:
def recode_borocd_counts(row):
    if row.community_board[0] != 'U':
      if 'MANHATTAN' in row.community_board:
        return '1' + row.community_board[:2]
      elif 'BRONX' in row.community_board:
        return '2' + row.community_board[:2]
      elif 'BROOKLYN' in row.community_board:
        return '3' + row.community_board[:2]
      elif 'QUEENS' in row.community_board:
        return '4' + row.community_board[:2]
      elif 'STATEN ISLAND' in row.community_board:
        return '5' + row.community_board[:2]
    else:
      return 'Invalid borocd'

df['borocd'] = df.apply(recode_borocd_counts, axis=1)

In [0]:

# this function will recode the community_board value into borocd in a new column
  def recode_borocd_counts(row):
    if row.community_board[0] != 'U':
      if 'MANHATTAN' in row.community_board:
        return '1' + row.community_board[:2]
      elif 'BRONX' in row.community_board:
        return '2' + row.community_board[:2]
      elif 'BROOKLYN' in row.community_board:
        return '3' + row.community_board[:2]
      elif 'QUEENS' in row.community_board:
        return '4' + row.community_board[:2]
      elif 'STATEN ISLAND' in row.community_board:
        return '5' + row.community_board[:2]
    else:
      return 'Invalid borocd'

cb_counts['borocd'] = cb_counts.apply(recode_borocd_counts, axis=1)

In [18]:
merged_data = pd.merge(left=cb_counts, right=df, left_on=['borocd','community_board'], right_on=['borocd','community_board'])
#checking column names
merged_data.head()

,community_board,count_of_311_requests,borocd,Unnamed: 0,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,landmark,facility_type,status,due_date,resolution_description,resolution_action_updated_date,bbl,borough,x_coordinate_(state_plane),y_coordinate_(state_plane),open_data_channel_type,park_facility_name,park_borough,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,0 Unspecified,39385,None,10,39886684,08/01/2018 12:03:55 AM,08/01/2018 01:22:36 PM,NYPD,New York City Police Department,noise - street/sidewalk,Loud Music/Party,Street/Sidewalk,NaN,240 FLATBUSH AVENUE,FLATBUSH AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Precinct,Closed,08/01/2018 08:03:55 AM,The Police Department responded to the complai...,08/01/2018 01:22:36 PM,NaN,Unspecified,NaN,NaN,MOBILE,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0 Unspecified,39385,None,24,39880125,08/01/2018 12:07:26 AM,08/01/2018 12:08:22 AM,HRA,HRA Benefit Card Replacement,benefit card replacement,Medicaid,Address Outside of NYC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed,NaN,The Human Resources Administration received yo...,NaN,NaN,Unspecified,NaN,NaN,ONLINE,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0 Unspecified,39385,None,238,39887282,08/01/2018 01:42:01 AM,08/14/2018 02:42:12 PM,DPR,Department of Parks and Recreation,maintenance or facility,Structure - Outdoors,Park,10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEW YORK,NaN,NaN,Closed,08/15/2018 01:42:01 AM,The Department of Parks and Recreation has com...,08/14/2018 02:42:12 PM,NaN,Unspecified,NaN,NaN,PHONE,First Park,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0 Unspecified,39385,None,262,39879188,08/01/2018 01:54:36 AM,08/01/2018 01:55:09 AM,HRA,HRA Benefit Card Replacement,benefit card replacement,Medicaid,NYC Street Address,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed,NaN,The Human Resources Administration received yo...,NaN,NaN,Unspecified,NaN,NaN,ONLINE,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0 Unspecified,39385,None,270,39880610,08/01/2018 01:58:27 AM,08/01/2018 03:06:18 PM,DOT,Department of Transportation,street sign - missing,St Name - Attached to Pole,Street,NaN,HORACE HARDING EXPRESSWAY,HORACE HARDING EXPRESSWAY,254 STREET,NaN,254 STREET,NaN,NaN,NaN,NaN,NaN,Closed,08/31/2018 01:58:27 AM,The Department of Transportation inspected the...,08/01/2018 03:06:18 PM,NaN,Unspecified,NaN,NaN,ONLINE,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
merged_data_noise = merged_data[merged_data.complaint_type=='noise - residential']
merged_data_noise = merged_data_noise.groupby('community_board').size().reset_index(name='count_of_311_requests')
#checking code
print(len(merged_data_noise))

69


In [20]:
merged_data_heat = merged_data[merged_data.complaint_type=='heat/hot water']
merged_data_heat = merged_data_heat.groupby('community_board').size().reset_index(name='count_of_311_requests')
#checking code
print(len(merged_data_heat))

66


In [21]:
merged_data_else = merged_data[merged_data.complaint_type != 'noise - residential'][merged_data.complaint_type != 'heat/hot water']
merged_data_else = merged_data_else.groupby('community_board').size().reset_index(name='count_of_311_requests')
#checking code
print(len(merged_data_else))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


77


In [22]:
# per the starter template, checking all
merged_data_all = merged_data.groupby('community_board').size().reset_index(name='count_of_311_requests')
#checking code
print(len(merged_data_all))

77


**Step 2.** Create a stacked bar chart comprised of three stacked bars. The total height of each bar (the sum of the three stacked bars) should reflect the total count of 311 complaints per CD.

1. number of 'Noise - Residential' complaints per CD
2. number of 'HEAT/HOT WATER' complaints per CD
3. number of other complaints per CD

In [0]:
# list of bar positions on x axis
position_on_x = np.arange(len(cd_data))

# original set of counts data
# note: it's important to use np.array(series) to convert the pandas series into an array. 
# if you don't do this, the stacking won't work corectly
counts = np.array(cd_data.sort_values(by='count_of_311_requests', ascending=False).count_of_311_requests)

# creating an array (aka list) of test data so we can easily visualize how the stacking works
test_counts = np.full(len(cd_data),20000)

plt.figure(figsize=(15, 6))

# need to name the plots as objects in order to access their color attributes in the legend
plot1 = plt.bar(
  position_on_x,
  test_counts,
  align='center',
  color='blue',
  label='Test Count'
)
# the second plot gets rendered on top of the first
plot2 = plt.bar(
  position_on_x,
  counts,
  bottom=test_counts, 
  # bottom defines the starting number to which each y value will be added instead of starting at 0
  # the counts value will get added to the corresponding counts_plus_10000 value
  align='center',
  color='cyan',
  label='Original Count'
)
plt.xticks(position_on_x, cd_data.sort_values(by='count_of_311_requests', ascending=False).borocd, rotation=90)

plt.title('Count of 311 Requests per Community District')
plt.xlabel('BoroCD')
plt.ylabel('# of Requests')
plt.legend() # defaults to putting legend in top right

plt.show()